In [ ]:
# Core scientific stack
import torch
import os

# Workaround for MKL/OpenMP duplication on some Windows/Conda setups.
# If you see warnings about duplicate OpenMP libraries, keeping this line avoids crashes.
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Library under test: optiprop provides optical elements and propagation methods
import optiprop

# NumPy for array helpers and simple numerical operations
import numpy as np

# Standard library utilities
import os

# OpenCV for reading DOE grayscale images
import cv2

In [ ]:
# Configuration
# - device: 'cpu' or 'cuda' (if you have a GPU and CUDA-enabled PyTorch installed)
# - dtype: float32 is memory-friendly; float64 increases precision
device = 'cpu'
dtype = torch.float32

# Near-field grid definition
# - NearField_pixel_size: sampling interval (m) on the near-field plane
# - NearField_field_Lx/Ly: physical size (m) of the simulated field in X and Y
NearField_pixel_size = 500e-9
NearField_field_Lx, NearField_field_Ly = 2000e-6, 2000e-6

# Create the near-field grid on which phase elements will be defined
Input_field = optiprop.NearField(
    pixel_size=NearField_pixel_size,
    field_Lx=NearField_field_Lx,
    field_Ly=NearField_field_Ly,
    dtype=dtype,
    device=device,
)

# Pretty print grid details (size, pixel pitch, device, etc.)
Input_field.rich_print()

In [ ]:
# VCSEL (Gaussian-like) source settings
# - wavelength: design wavelength (m)
# - divergence_angle: full-angle divergence in degrees (approximation)
wavelength = 0.940e-6
VSCEL_element = optiprop.VSCELPhase(Input_field)
VSCEL_element.calculate_phase(
    design_lambda=wavelength,
    divergence_angle=24,
    vscel_center=[0, 0],
)

# Display parameters and visualize the source field
VSCEL_element.rich_print()
VSCEL_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=[-5, 5],
    ylim=[-5, 5],
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
# Define propagation distance for free-space propagation of VCSEL beam
propagation_distance = 1000e-6

# Create an ASM propagator for near-to-far propagation
propagator = optiprop.ASMPropagation(
    propagation_wavelength=wavelength,
    propagation_distance=propagation_distance,
    n=1,
    device=device,
    dtype=dtype,
)

# Feed the VCSEL near-field into the propagator
propagator.set_input_field(
    u_in=VSCEL_element.U0,
    pixel_size=NearField_pixel_size,
)

# Optionally inspect input field settings
# propagator.show_input_U(cmap='turbo', fontsize=24, xlim=None, ylim=None, dark_style=True, show=True)

# Propagate and visualize complex field at the observation plane
propagator.propagate()
propagator.show_output_U(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
# Show intensity distribution (wide field-of-view) to see the beam footprint
propagator.show_intensity(
    cmap='CMRmap',
    fontsize=24,
    xlim=[-1000, 1000],
    ylim=[-1000, 1000],
    clim=None,
    unit='µm',
    norm='linear',
    dark_style=True,
    show=True,
)

In [ ]:
# Add a focusing metalens element to collimate/focus the VCSEL beam
lens_diameter = 1500e-6
Metalens_element = optiprop.EqualPathPhase(Input_field)
Metalens_element.calculate_phase(
    focal_length=1000e-6,
    design_lambda=wavelength,
    lens_diameter=lens_diameter,
    lens_center=[0, 0],
    aperture_type='circle',
    aperture_size=[lens_diameter],
)

# Inspect parameters and visualize the phase profile
Metalens_element.rich_print()
Metalens_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
#DOE setting
lens_diameter = 1500e-6

DOE_filepath = 'DOE\\W808_P5040_3x3.png'
img = cv2.imread(DOE_filepath, cv2.IMREAD_GRAYSCALE)
unit_cell = torch.from_numpy(img).float()  # shape: [H, W], dtype=float32
unit_cell = unit_cell/torch.max(unit_cell)

DOE_element = optiprop.DiffractiveOpticsElement(Input_field)
DOE_element.calculate_phase(
    unit_cell=unit_cell,
    xperiod=5040e-9,
    yperiod=5040e-9,
    aperture_type='circle',
    aperture_size=[lens_diameter]
)

DOE_element.rich_print()
DOE_element.draw(
    cmap='turbo', 
    fontsize=24, 
    xlim=None, 
    ylim=None, 
    selected_field='all',
    dark_style=True,
    show=True
)

DOE_element.draw(
    cmap='turbo', 
    fontsize=24, 
    xlim=[-50, 50], 
    ylim=[-50, 50], 
    selected_field='all',
    dark_style=True,
    show=True
)

In [ ]:
# VSCEL source passthrough meta-optics element
VSCEL_Source = propagator.get_output_U
# meta-optics element : Metalens + DOE
U_MOE = Metalens_element.U0 * DOE_element.U0
# total field
U_total = VSCEL_Source * U_MOE
Input_field.input_field(U_total)
Input_field.draw(
    U_total,
    cmap='turbo', 
    fontsize=24, 
    xlim=None, 
    ylim=None, 
    selected_field='all',
    dark_style=True,
    show=True
)

In [ ]:
propagation_distance = 0.7
propagator = optiprop.FresnelPropagation(
    propagation_wavelength=wavelength, 
    propagation_distance=propagation_distance, 
    n=1,
    device=device,
    dtype=dtype
)


propagator.set_input_field(
    u_in = U_total, 
    pixel_size = NearField_pixel_size,
    )

propagator.show_input_U(
    cmap='turbo', 
    fontsize=24, 
    xlim=None, 
    ylim=None, 
    dark_style=True,
    show=True
)

propagator.set_output_field(
    output_pixel_size=0.1e-3,
)
# xy far field plane setting
# propagator.show_output_U(
#     cmap='turbo', 
#     fontsize=24, 
#     xlim=None, 
#     ylim=None, 
#     selected_field='all',
#     dark_style=True,
#     show=True
# )


In [ ]:
propagator.propagate()
propagator.show_intensity(
    cmap='turbo', 
    fontsize=24, 
    xlim=[-3 , 3], 
    ylim=[-3 , 3], 
    clim=None, 
    unit='mm', 
    norm='linear',
    dark_style=True,
    show=True
)

In [ ]:
propagator.show_intensity(
    cmap='turbo', 
    fontsize=24, 
    xlim=[-200, 200], 
    ylim=[-200, 200], 
    clim=[1e-12, 1e-7], 
    unit='mm', 
    norm='log',
    dark_style=True,
    show=True
)